In [1]:
!pip install bitsandbytes
!pip install transformers

In [2]:
import pandas as pd
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from huggingface_hub import notebook_login
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
!pip install accelerate

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
csv_path = '/content/drive/MyDrive/IR-Project/res/csv_etl_files'
csv_data = os.listdir(csv_path)

In [6]:
!pip install rank_bm25

In [7]:
import sys

# Assuming your file is named 'your_module.py' and is uploaded to the root of Colab's session storage
file_path = '/content/drive/MyDrive/IR-Project/src'
if file_path not in sys.path:
    sys.path.append(file_path)

from document_retrieval_BM25 import get_query_results

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
# main_model_name = 'tomrb/bettercallbloom-3b'
# tokenizer = AutoTokenizer.from_pretrained(main_model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     main_model_name,
#     device_map="auto",
#     trust_remote_code=True,
#     quantization_config=bnb_config
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def prompt_generation(ranked_text,query):
    context = ""
    for i in range(len(ranked_text)):
        context += f"{i+1}: {ranked_text[i]}"
    prompt = f"""Generate legal advice for {query} using the following contexual information {context}"""
    return prompt

In [ ]:
def generate_text(prompt,tokenizer,model):
    """
    Generate text from a pre-trained language model given a prompt and a model name.

    Parameters:
    prompt (str): The prompt text to feed to the language model.
    model_name (str): The model identifier on Hugging Face's model hub.

    Returns:
    str: The text generated by the model.
    """

    # Encode the prompt text
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    print("Done")
    # Generate text using the model
    output = model.generate(
    input_ids,
    max_length=250,
    max_new_tokens=250,
    temperature=0.5,  # Adjust temperature for determinism
    top_p=0.95,       # Narrow down while allowing some diversity
    no_repeat_ngram_size=2  # Prevent repeating n-grams
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("Generate legal advice for")[1].strip()  # Simplified example
    return cleaned_text

In [16]:
def prompt2_generation(query,ranked_text):
  context = ""
  for i in range(len(ranked_text)):
      context += f"{i+1}: {ranked_text[i]}"

  prompt2 = f"""Given the a legal advice for this query: {query}
              using the following contexual information {context}"""
  return prompt2

In [10]:
def generate_text2(prompt,tokenizer,model):
    """
    Generate text from a pre-trained language model given a prompt and a model name.

    Parameters:
    prompt (str): The prompt text to feed to the language model.
    model_name (str): The model identifier on Hugging Face's model hub.

    Returns:
    str: The text generated by the model.
    """

    # Encode the prompt text
    lst = [prompt]
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input = tokenizer(lst, return_tensors='pt').to(device)
    print("Done")
    # Generate text using the model
    output = model.generate(
    **input,
    max_length=250,
    max_new_tokens=250,
    temperature=0.5,  # Adjust temperature for determinism
    top_p=0.95,       # Narrow down while allowing some diversity
    no_repeat_ngram_size=2  # Prevent repeating n-grams
    )

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    #cleaned_text = generated_text.split("Generate legal advice for")[1].strip()  # Simplified example
    return generated_text

In [11]:
model_path = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [14]:
query1 = "What is goods and service Tax?"
results1 = get_query_results(query1,3)
query2 = "What is coal mine allocation act?"
results2 = get_query_results(query2,3)
query3 = "What is service charge?"
results3 = get_query_results(query3,3)
query4 = "What is existing national waterways act?"
results4 = get_query_results(query4,3)
query5 = "Act that was passed for establishment for agricultural university?"
results5 = get_query_results(query5,3)
query6 = "Explain bank notes"
results6 = get_query_results(query6,3)

/content/drive/MyDrive/IR-Project/res/csv_etl_files
Files present /content/drive/MyDrive/IR-Project/res/csv_etl_files :


In [20]:
prompt2 = prompt2_generation(query1,results1['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Done
Given the a legal advice for this query: What is goods and service Tax?
              using the following contexual information 1: shall not be void if it is made for adequate consideration, in good faith and without notice of the pendency of such proceedings under this Act or without notice of such tax or other sum payable by the said person, or with the previous permission of the proper officer.2: 39. Protection ofaction taken in good faith.3: if any, shall not be admissible as input tax credit under this Act.(9)) Save as otherwise provided in this Chapter, the goods or services or both supplied on or after the appointed day in pursuance of acontract entered into prior to the appointed day shall be liable to tax under the provisions of this Act.(10) (a)) Notwithstanding anything contained in section 12 of the Central Goods and Services TaxAct, no tax shall be payable on goods under this Act to the extent the tax was leviable on the said good sunder the existing law.13 (b)) The p

In [21]:
prompt2 = prompt2_generation(query2,results2['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Done
Given the a legal advice for this query: What is coal mine allocation act?
              using the following contexual information 1: third part y,, subsequent to such allocation of Scheduled Icoal mines, then, the third party shall be deemed to be the priorallottee;(o)) ―Schedule ‖ means aSchedule appended to this Act;(p)) ―Schedule Icoal mines ‖ means, —(i)) all the coal mines and coal blocks the allocation of which was cancelled by the judgment dated 25th August, 2014 and its order dated 24th September, 2014 passed in Writ Petition(Criminal) No.120 of 2012, including those allotments which may have been de -allocated prior to and during the pend ency of the said Writ2: 33. Repeal and saving.SCHEDULE I..SCHEDULE II.SCHEDULE III.SCHEDULE IV.3THE COAL MINES (SPECIAL PROVISIONS) ACT, 2015ACT NO. 11 OF 2015[30th March , 2015.]An Act to provide for allocation of coal mines and vesting of the right, title and interest in and over the land and mine infrastructure together with mining l

In [26]:
prompt2 = prompt2_generation(query3,results3['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Done
Given the a legal advice for this query: What is service charge? 
              using the following contexual information 1: supply clearly identifies the service in question and its supplier in non-taxable territory;(b)) the intermediary involved in the supply does not authorise the charge to the customer or take part in its charge which is that the intermediary neither collects or processes payment in any manner nor is responsible for the payment between the non-taxable online recipient and the supplier of suchservices;(c)) the intermediary involved in the supply does not authorise delivery; and(d)) the general terms and conditions of the supply are not set by the2: 

In [23]:
prompt2 = prompt2_generation(query4,results4['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Done
Given the a legal advice for this query: What is existing national waterways act?
              using the following contexual information 1: 4.. Amendment of section 2of Act 82 of 1985 .—In the Inland Waterways Authority of IndiaAct, 1985 , in section 2,, for clause ( h)), the following clause shall be substituted, namely: —„(h)) “national waterway ” means the inland waterway declared by section 2of the National Waterways Act, 2016, to be anational waterway.Explanation. —If Parliament declares by law any other waterway to be anational waterway, then,from the date on which such declaration takes effect, such other waterway —(i)) shall also be deemed to be anational waterway within2: salvage services including special compensation relating thereto;(d)) claims for port, canal, and other waterway dues and pilotage dues and any other statutory dues related to the vessel;(e)) claims based on tort arising out of loss or damage caused by the operation of the vessel other than loss or dama

In [24]:
prompt2 = prompt2_generation(query5,results5['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Done
Given the a legal advice for this query: Act that was passed for establishment for agricultural university?
              using the following contexual information 1: establishment under sub -section ( 15), to the Authority within the specified period.(17) The Authority shall on being satisfied that the mental health establishment fulfils the specified minimum standards for registration, grant permanent certificate of registration to such establishment.(18) The Authority shall, within aperiod of forty -five days after the expiry of the period specified under this section, pass an order, either —(a)) grant permanent certificate of registration; or(b)) reject the application after recording the reasons2: dairy science, fisheries, forestry including farm forestry, home -science, agricultural engine ering and technology, marketing and processing of agricultural and animal husbandry products, land use and management;(d)) “Board ” means the Board of Management of the University;(e)) “Bo

In [25]:
prompt2 = prompt2_generation(query6,results6['retrieval_results'])
final_text = generate_text2(prompt2,tokenizer,model)
print(final_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=250) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Done
Given the a legal advice for this query: Explain bank notes
              using the following contexual information 1: bank note” means abank note of the denominational value of five hundred rupees or one thousand rupees of the series existing on or before the 8th day of November, 2016.(2))The words and expressions used and not defined in this Act but defined in the Reserve Bank of India Act, 1934 (2of 1934) or the Banking Regulation Act, 1949 (10 of 1949) shall have the meanings respectively assigned to them in those Acts.2: 2.. Subs. by s.. 15, ibid., for ―acredit note ‖ (w..e..f.. 1--2--2019).3: credit notes for supplies made in afinancial year ] containing such particulars as may be prescribed.(4)) Any registered person who issues adebit note in relation to asupply of goods or service sor both shall declare the details of such debit note in the return for the month during which such debit note has been issued and the tax liability shall be adjusted in such manner as may be pre